In [ ]:
import numpy as np
import pandas as pd
from pybaseball import statcast
from datetime import datetime
import os
import pickle

In [ ]:
def is_nan(val):
    return 0 if pd.isna(val) else 1
def get_year_pitches(year):
    data = statcast(f"{year}-01-01",f"{year}-12-31")
    return data
    
def score_game_re288(game,matrix_frame):
    innings = []
    t = game[['inning', 'inning_topbot']].drop_duplicates().sort_values(by=['inning', 'inning_topbot'], ascending =[True,False]).reset_index(drop=True)
    for i in t.index:
        inning_frame = game.loc[(game['inning'] == t.iloc[i]['inning']) & 
                                  (game['inning_topbot'] == t.iloc[i]['inning_topbot'])].sort_values(by=['at_bat_number', 'pitch_number'], ascending =[True,True]).reset_index(drop = True)
        for j in inning_frame.index:
            try:
                initial_state = (f"{is_nan(inning_frame.iloc[j]['on_3b'])}-{is_nan(inning_frame.iloc[j]['on_2b'])}-{is_nan(inning_frame.iloc[j]['on_1b'])}"
                    ,f"{inning_frame.iloc[j]['balls']}-{inning_frame.iloc[j]['strikes']}-{inning_frame.iloc[j]['outs_when_up']}")
                initial_exp = matrix_frame.loc[initial_state[0],initial_state[1]]
                if j >= max(inning_frame.index):
                    final_state = ("0-0-0","0-0-3")
                    final_exp = 0
                else:
                    final_state = (f"{is_nan(inning_frame.iloc[j+1]['on_3b'])}-{is_nan(inning_frame.iloc[j+1]['on_2b'])}-{is_nan(inning_frame.iloc[j+1]['on_1b'])}"
                    ,f"{inning_frame.iloc[j+1]['balls']}-{inning_frame.iloc[j+1]['strikes']}-{inning_frame.iloc[j+1]['outs_when_up']}") 
                    final_exp = matrix_frame.loc[final_state[0],final_state[1]]
            except KeyError:
                continue
            inning_frame.loc[j,'run_exp'] = final_exp - initial_exp
        innings.append(inning_frame)
    edited_game_frame = pd.concat(innings, ignore_index = True)
    return edited_game_frame
            
                
            
def score_year_re288(year,matrix_frame):
    i = 0
    game_pks = pd.unique(year.game_pk)
    concat_frame = pd.DataFrame()
    for game in game_pks:
        i+=1 
        if round(i/len(game_pks) * 100) % 20 == 0:
            print(i/len(game_pks) * 100)
        game_frame = year.loc[year['game_pk']==game]
        game_scored = score_game_re288(game_frame,matrix_frame)
        concat_frame = pd.concat([concat_frame,game_scored],ignore_index = True)
    return concat_frame
        

In [ ]:
frame_pickled= pickle.load(open('run_expectency_frames.txt','rb'))

run_exp ={}
for x in [2015,2016,2017,2018,2019,2021]:
    run_exp[x] = score_year_re288(get_year_pitches(x),frame_pickled[x])
    
    
#this took a while to run, so I ran it overnight and saved the objects in a pickled file to pick back up later
with open('mlb_pitches_scored_re288.txt','wb') as retext:
    pickle.dump(run_exp,retext)
    


In [ ]:
with open('mlb_pitches_scored_re288.txt','wb') as retext:
    pickle.dump(run_exp,retext)
frame_pickled= pickle.load(open('mlb_pitches_scored_re288.txt','rb'))

In [ ]:

frame_pickled= pickle.load(open('mlb_pitches_scored_re288.txt','rb'))

In [ ]:
frame_pickled[2021]